libraries

In [ ]:
using UniversalDiffEq, DataFrames, CSV, Plots, Serialization

function to plot phase planes

In [ ]:
# visualize 2 variables (e.g., Coral vs Macro) by fixing the third variable (e.g., Turf) to be 1 - Coral - Macro
function phase_plane(model; start = 0.05, stop = 1.0, step = 0.1, max_T = 250, 
                     title = "Plot Title", xlabel = "Macroalgae Cover", ylabel = "Coral Cover")
    
    u0_array = Vector{Vector{Float64}}()
    for macro_ in start:step:stop
        for coral in start:step:stop
            if (coral + macro_ > 1)
                continue 
            end
            turf = 1.0 - coral - macro_
            #push!(u0_array, [coral, macro_, turf]) 
            push!(u0_array, [coral, macro_])
        end 
    end 

    plt = UniversalDiffEq.phase_plane(model, u0_array, T = max_T, idx = [2, 1])
    #title!(plt, title)
    xlabel!(plt, xlabel)
    ylabel!(plt, ylabel)
    return plt 
end

Varying the IC of coral from 0.05-0.85 with g = 0.1 and process noise on g 
Because grazing is so low, we expect only one state to be detected

Coral IC = 0.05

In [ ]:
# Read in data
df = CSV.read("data/Noise on g01/Mumby_coralIC005_model_output_noiseong_g01.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.875, 0.0, "stable_node"),         # (M=0.875, C=0.0)
    (0.0,   0.56, "unstable_saddle"),    # (M=0.0, C=0.56)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.15

In [ ]:
# Read in data
df = CSV.read("data/Noise on g01/Mumby_coralIC015_model_output_noiseong_g01.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.875, 0.0, "stable_node"),         # (M=0.875, C=0.0)
    (0.0,   0.56, "unstable_saddle"),    # (M=0.0, C=0.56)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.25

In [ ]:
# Read in data
df = CSV.read("data/Noise on g01/Mumby_coralIC025_model_output_noiseong_g01.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.875, 0.0, "stable_node"),         # (M=0.875, C=0.0)
    (0.0,   0.56, "unstable_saddle"),    # (M=0.0, C=0.56)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.35

In [ ]:
# Read in data
df = CSV.read("data/Noise on g01/Mumby_coralIC035_model_output_noiseong_g01.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    #title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.875, 0.0, "stable_node"),         # (M=0.875, C=0.0)
    (0.0,   0.56, "unstable_saddle"),    # (M=0.0, C=0.56)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.42

In [ ]:
# Read in data
df = CSV.read("data/Noise on g01/Mumby_coralIC042_model_output_noiseong_g01.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.875, 0.0, "stable_node"),         # (M=0.875, C=0.0)
    (0.0,   0.56, "unstable_saddle"),    # (M=0.0, C=0.56)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.44

In [ ]:
# Read in data
df = CSV.read("data/Noise on g01/Mumby_coralIC044_model_output_noiseong_g01.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    #title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.875, 0.0, "stable_node"),         # (M=0.875, C=0.0)
    (0.0,   0.56, "unstable_saddle"),    # (M=0.0, C=0.56)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.45

In [ ]:
# Read in data
df = CSV.read("data/Noise on g01/Mumby_coralIC045_model_output_noiseong_g01.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.875, 0.0, "stable_node"),         # (M=0.875, C=0.0)
    (0.0,   0.56, "unstable_saddle"),    # (M=0.0, C=0.56)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.55

In [ ]:
# Read in data
df = CSV.read("data/Noise on g01/Mumby_coralIC055_model_output_noiseong_g01.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.875, 0.0, "stable_node"),         # (M=0.875, C=0.0)
    (0.0,   0.56, "unstable_saddle"),    # (M=0.0, C=0.56)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.65

In [ ]:
# Read in data
df = CSV.read("data/Noise on g01/Mumby_coralIC065_model_output_noiseong_g01.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.875, 0.0, "stable_node"),         # (M=0.875, C=0.0)
    (0.0,   0.56, "unstable_saddle"),    # (M=0.0, C=0.56)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.75

In [ ]:
# Read in data
df = CSV.read("data/Noise on g01/Mumby_coralIC075_model_output_noiseong_g01.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.875, 0.0, "stable_node"),         # (M=0.875, C=0.0)
    (0.0,   0.56, "unstable_saddle"),    # (M=0.0, C=0.56)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.85

In [ ]:
# Read in data
df = CSV.read("data/Noise on g01/Mumby_coralIC085_model_output_noiseong_g01.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.875, 0.0, "stable_node"),         # (M=0.875, C=0.0)
    (0.0,   0.56, "unstable_saddle"),    # (M=0.0, C=0.56)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Varying the IC of coral from 0.05-0.85 with noise on g and g = 0.3 to ID what states the NN detects 

Coral IC = 0.05

In [ ]:
# Read in data
df = CSV.read("data/Noise on g03/Mumby_coralIC005_model_output_noiseong_g03.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)


In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)
plot_state_estimates(model)


In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M =0, C=0.56)
    (0.28, 0.25, "unstable_saddle"),    # (M=0.28, C=0.25)
    (0.625, 0.0, "stable_node"),         # (M=0.625, C=0)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)


Coral IC = 0.15

In [ ]:
# Read in data
df = CSV.read("data/Noise on g03/Mumby_coralIC015_model_output_noiseong_g03.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M =0, C=0.56)
    (0.28, 0.25, "unstable_saddle"),    # (M=0.28, C=0.25)
    (0.625, 0.0, "stable_node"),         # (M=0.625, C=0)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)


Coral IC = 0.25

In [ ]:
# Read in data
df = CSV.read("data/Noise on g03/Mumby_coralIC025_model_output_noiseong_g03.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M =0, C=0.56)
    (0.28, 0.25, "unstable_saddle"),    # (M=0.28, C=0.25)
    (0.625, 0.0, "stable_node"),         # (M=0.625, C=0)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)


Coral IC = 0.35

In [ ]:
# Read in data
df = CSV.read("data/Noise on g03/Mumby_coralIC035_model_output_noiseong_g03.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M =0, C=0.56)
    (0.28, 0.25, "unstable_saddle"),    # (M=0.28, C=0.25)
    (0.625, 0.0, "stable_node"),         # (M=0.625, C=0)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)


Coral IC = 0.42

In [ ]:
# Read in data
df = CSV.read("data/Noise on g03/Mumby_coralIC042_model_output_noiseong_g03.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M =0, C=0.56)
    (0.28, 0.25, "unstable_saddle"),    # (M=0.28, C=0.25)
    (0.625, 0.0, "stable_node"),         # (M=0.625, C=0)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)


Coral IC = 0.44

In [ ]:
# Read in data
df = CSV.read("data/Noise on g03/Mumby_coralIC044_model_output_noiseong_g03.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M =0, C=0.56)
    (0.28, 0.25, "unstable_saddle"),    # (M=0.28, C=0.25)
    (0.625, 0.0, "stable_node"),         # (M=0.625, C=0)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)


Coral IC = 0.45

In [ ]:
# Read in data
df = CSV.read("data/Noise on g03/Mumby_coralIC045_model_output_noiseong_g03.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M =0, C=0.56)
    (0.28, 0.25, "unstable_saddle"),    # (M=0.28, C=0.25)
    (0.625, 0.0, "stable_node"),         # (M=0.625, C=0)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)


Coral IC = 0.55

In [ ]:
# Read in data
df = CSV.read("data/Noise on g03/Mumby_coralIC055_model_output_noiseong_g03.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M =0, C=0.56)
    (0.28, 0.25, "unstable_saddle"),    # (M=0.28, C=0.25)
    (0.625, 0.0, "stable_node"),         # (M=0.625, C=0)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)


Coral IC = 0.65

In [ ]:
# Read in data
df = CSV.read("data/Noise on g03/Mumby_coralIC065_model_output_noiseong_g03.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M =0, C=0.56)
    (0.28, 0.25, "unstable_saddle"),    # (M=0.28, C=0.25)
    (0.625, 0.0, "stable_node"),         # (M=0.625, C=0)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)


Coral IC = 0.75

In [ ]:
# Read in data
df = CSV.read("data/Noise on g03/Mumby_coralIC075_model_output_noiseong_g03.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M =0, C=0.56)
    (0.28, 0.25, "unstable_saddle"),    # (M=0.28, C=0.25)
    (0.625, 0.0, "stable_node"),         # (M=0.625, C=0)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)


Coral IC = 0.85

In [ ]:
# Read in data
df = CSV.read("data/Noise on g03/Mumby_coralIC085_model_output_noiseong_g03.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M =0, C=0.56)
    (0.28, 0.25, "unstable_saddle"),    # (M=0.28, C=0.25)
    (0.625, 0.0, "stable_node"),         # (M=0.625, C=0)
    (0.0,   0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)


Varying the IC of coral from 0.05-0.85 with noise on g and g = 0.5 to ID what states the NN detects
Because g is relatively high, it should only be one state -- the coral dominate state

Coral IC = 0.05

In [ ]:
# Read in data
df = CSV.read("data/Noise on g05/Mumby_coralIC005_model_output_noiseong_g05.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)
plot_state_estimates(model)


In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M=0, C=0.56)
    (0.375, 0.0, "unstable_saddle"),    # (M=0.375, C=0.0)
    (0.0, 0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.15

In [ ]:
# Read in data
df = CSV.read("data/Noise on g05/Mumby_coralIC015_model_output_noiseong_g05.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M=0, C=0.56)
    (0.375, 0.0, "unstable_saddle"),    # (M=0.375, C=0.0)
    (0.0, 0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.25

In [ ]:
# Read in data
df = CSV.read("data/Noise on g05/Mumby_coralIC025_model_output_noiseong_g05.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M=0, C=0.56)
    (0.375, 0.0, "unstable_saddle"),    # (M=0.375, C=0.0)
    (0.0, 0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.35

In [ ]:
# Read in data
df = CSV.read("data/Noise on g05/Mumby_coralIC035_model_output_noiseong_g05.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M=0, C=0.56)
    (0.375, 0.0, "unstable_saddle"),    # (M=0.375, C=0.0)
    (0.0, 0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.42

In [ ]:
# Read in data
df = CSV.read("data/Noise on g05/Mumby_coralIC042_model_output_noiseong_g05.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M=0, C=0.56)
    (0.375, 0.0, "unstable_saddle"),    # (M=0.375, C=0.0)
    (0.0, 0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.44

In [ ]:
# Read in data
df = CSV.read("data/Noise on g05/Mumby_coralIC044_model_output_noiseong_g05.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M=0, C=0.56)
    (0.375, 0.0, "unstable_saddle"),    # (M=0.375, C=0.0)
    (0.0, 0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.45

In [ ]:
# Read in data
df = CSV.read("data/Noise on g05/Mumby_coralIC045_model_output_noiseong_g05.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M=0, C=0.56)
    (0.375, 0.0, "unstable_saddle"),    # (M=0.375, C=0.0)
    (0.0, 0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.55

In [ ]:
# Read in data
df = CSV.read("data/Noise on g05/Mumby_coralIC055_model_output_noiseong_g05.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M=0, C=0.56)
    (0.375, 0.0, "unstable_saddle"),    # (M=0.375, C=0.0)
    (0.0, 0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.65

In [ ]:
# Read in data
df = CSV.read("data/Noise on g05/Mumby_coralIC065_model_output_noiseong_g05.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M=0, C=0.56)
    (0.375, 0.0, "unstable_saddle"),    # (M=0.375, C=0.0)
    (0.0, 0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.75

In [ ]:
# Read in data
df = CSV.read("data/Noise on g05/Mumby_coralIC075_model_output_noiseong_g05.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M=0, C=0.56)
    (0.375, 0.0, "unstable_saddle"),    # (M=0.375, C=0.0)
    (0.0, 0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)

Coral IC = 0.85

In [ ]:
# Read in data
df = CSV.read("data/Noise on g05/Mumby_coralIC085_model_output_noiseong_g05.csv", DataFrame)

# Extract the columns you want as the state variables 
times = df.Time
observations = Array(df[:, ["Coral", "Macro"]])'

# Prepare data for model: DataFrame with all relevant columns
df_model = df[:, ["Time", "Coral", "Macro"]]


In [ ]:
plot(df.Time, df.Coral, xlabel = "Time", ylabel="Coral", label="", lw=2)

In [ ]:
plot(df.Time, df.Macro, xlabel = "Time", ylabel="Macro", label="", lw=2)

In [ ]:
# Estimate some rates as constants
NN, params = UniversalDiffEq.SimpleNeuralNetwork(2, 1)
init_parameters = (NN = params, r1 = 0.8, r2 = 0.3, r3 = 0.1, r4 =0.5)
function dudt(u, p, t)
    #r = NN(u, p.NN)
    r = exp.(NN(u,p.NN)) # bounding rates to positive values
	Coral,MacroAlg = u
    Turf = 1 - Coral - MacroAlg
    dC = p.r1*Coral*Turf - p.r2*Coral - p.r3*Coral*MacroAlg
    #dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - ((r[1]*MacroAlg)/(MacroAlg+Turf))
    dM = p.r3*Coral*MacroAlg + p.r4*MacroAlg*Turf - r[1]*MacroAlg
    return [dC,dM]
end

In [ ]:
model = CustomDerivatives(df_model, dudt, init_parameters)

In [ ]:
train!(model; 
    loss_function = "derivative matching",
    loss_options = (d=2),  
    optimizer = "ADAM",
    regularization_weight = 1e-4,#0.0,
    verbose = true,
    optim_options = (maxiter = 100, step_size = 0.01)
)

In [ ]:
plot_state_estimates(model)

In [ ]:
phasePlot = phase_plane(model;
    title = "Macro vs Coral Phase Plane",
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)
display(phasePlot)

In [ ]:
# Define phase plane

phasePlot = phase_plane(model;
    xlabel = "Macroalgae Cover",
    ylabel = "Coral Cover"
)

#    Equilibria taken from Mathematica, with stability labels
#    Mathematica gave (C, M); convert to Julia (M, C)

# Stability color mapping
stability_colors = Dict(
    "stable_focus"      => :green,
    "stable_node"       => :green,
    "unstable_saddle"   => :red,
    "unstable_repelling"=> :orange
)

# Equilibria to plot:
# (Macro, Coral, Stability)
eq_points = [
    (0.0, 0.56, "stable_node"),         # (M=0, C=0.56)
    (0.375, 0.0, "unstable_saddle"),    # (M=0.375, C=0.0)
    (0.0, 0.0,  "unstable_repelling")  # (M=0.0, C=0.0)
]

# Plot equilibria with stability color

for (M, C, label) in eq_points
    scatter!(phasePlot,
        [M], [C],
        markershape = :circle,
        markersize = 8,
        color = stability_colors[label],
        label = ""
    )
end

# Add legend entries (optional)

scatter!(phasePlot, [NaN], [NaN], color=:green,  label="Stable")
scatter!(phasePlot, [NaN], [NaN], color=:red,    label="Saddle")
scatter!(phasePlot, [NaN], [NaN], color=:orange, label="Repelling")

# Display plot -- with eqbm points overlayed
display(phasePlot)